In [1]:
# General
import sys
import os
import glob
from time import time
from collections import OrderedDict as od
from importlib import reload
from glob import glob
import itertools

# Scientific
import numpy as np
import pandas as pd
pd.options.display.max_rows = 200
pd.options.display.max_columns = 999
import scipy.io as sio

# Stats
import scipy.stats as stats
import statsmodels.api as sm
import astropy.stats.circstats as circstats
import pycircstat
import random
from sklearn.decomposition import PCA

# Plots
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import matplotlib as mpl
from matplotlib.lines import Line2D
import matplotlib.path as mpath
import matplotlib.patches as patches
mpl.rcParams['grid.linewidth'] = 0.1
mpl.rcParams['grid.alpha'] = 0.75
mpl.rcParams['lines.linewidth'] = 1
mpl.rcParams['lines.markersize'] = 3
mpl.rcParams['xtick.labelsize'] = 15
mpl.rcParams['ytick.labelsize'] = 15 
colors = ['1f77b4', 'd62728', '2ca02c', 'ff7f0e', '9467bd', 
          '8c564b', 'e377c2', '7f7f7f', 'bcbd22', '17becf']
mpl.rcParams['axes.prop_cycle'] = mpl.cycler('color', colors)
mpl.rcParams['axes.spines.right'] = False
mpl.rcParams['axes.spines.top'] = False
mpl.rcParams['axes.formatter.offset_threshold'] = 2
mpl.rcParams['axes.labelsize'] = 17
mpl.rcParams['axes.labelpad'] = 10
mpl.rcParams['axes.titlesize'] = 19
mpl.rcParams['axes.grid'] = False
mpl.rcParams['axes.axisbelow'] = True
mpl.rcParams['legend.loc'] = 'upper right'
mpl.rcParams['legend.fontsize'] = 15
mpl.rcParams['legend.frameon'] = False
mpl.rcParams['figure.dpi'] = 300
mpl.rcParams['figure.titlesize'] = 19
mpl.rcParams['figure.figsize'] = (6.85039, 4.79527) 
mpl.rcParams['figure.subplot.wspace'] = 0.25 
mpl.rcParams['figure.subplot.hspace'] = 0.25 
mpl.rcParams['font.sans-serif'] = ['Helvetica']
mpl.rcParams['savefig.format'] = 'pdf'
mpl.rcParams['pdf.fonttype'] = 42

# Neuro
import mne
from ptsa.data.TimeSeriesX import TimeSeries 
#import neurodsp

# Personal
sys.path.append('/home1/dscho/code/general')
sys.path.append('/home1/dscho/code/projects/manning_replication')
sys.path.append('/home1/dscho/code/projects/unit_activity_and_hpc_theta')
import data_io as dio
import array_operations as aop
import spectral_processing as spp
import manning_utils
import manning_analysis
import phase_locking
import lfp_synchrony
from eeg_plotting import plot_trace, plot_trace2

# Colors
n = 4
c = 2
colors = [sns.color_palette('Blues', n)[c], 
          sns.color_palette('Reds', n)[c], 
          sns.color_palette('Greens', n)[c],
          sns.color_palette('Purples', n)[c],
          sns.color_palette('Oranges', n)[c],
          sns.color_palette('Greys', n)[c],
          sns.color_palette('YlOrBr', n+3)[c],
          'k']
cmap = sns.palettes.blend_palette((colors[0], 
                                   sns.palettes.blend_palette((colors[0], 'w', colors[1]), 101)[45],
                                   sns.palettes.blend_palette((colors[0], 'w', colors[1]), 101)[55],
                                   colors[1]), 101)
rdbu_cut = sns.color_palette('RdBu_r', 500)[50:450]

labelfontsize = 12
tickfontsize = 12
figfontsize = 14
labfs = labelfontsize
tickfs = tickfontsize
figfs = figfontsize

/home1/dscho/anaconda3/envs/memlab/lib/python3.6/site-packages/ptsa/data/TimeSeriesX.py:7: DeprecationWarning: importing from ptsa.data.TimeSeries is deprecated; import from ptsa.data.timeseries instead
  DeprecationWarning)


In [2]:
# Get session info.
subj_df = phase_locking.get_subj_df()
subj_df.insert(4, 'chan_ind', subj_df.chan.apply(lambda x: int(x)-1))

freqs = np.array([2**((i/2) - 1) for i in range(16)])
sampling_rate = 2000
time_win=2

input_dir = '/scratch/dscho/unit_activity_and_hpc_theta/data/crosselec_phase_locking'

power_fname = os.path.join(input_dir, 'wavelet', 'power', 
                           'power-Z-log-{}-iChan{}-iFreq{}-2000Hz-notch60_120Hz-nospikeinterp-5cycles-16log10freqs_0.5_to_90.5Hz.pkl')
spike_fname = os.path.join(input_dir, 'spike_inds', 'spike_inds-2000Hz-{}-unit{}.pkl')
fr_fname = os.path.join(input_dir, 'spike_frs', 'spike_frs-2000Hz-{}-unit{}.pkl')
pl_fname = os.path.join(input_dir, 'phase_locking', 'unit_to_region', 
                        'phase_locking_stats-{}-unit_{}-lfp_{}-2000Hz-notch60_120Hz5cycles-16log10freqs_0.5_to_90.5Hz.pkl')

In [3]:
# Load phase-locking stats.
start_time = time()

if True:
    # Load the unit-to-region phase-locking DataFrame.
    pl_dir = '/scratch/dscho/unit_activity_and_hpc_theta/data/crosselec_phase_locking/phase_locking/unit_to_region'
    pl_fname = os.path.join(pl_dir, 'all_phase_locking_stats-14026_unit_to_region_pairs-2000Hz-notch60_120Hz-5cycles-16log10freqs_0.5_to_90.5Hz.pkl')
    pl_df = phase_locking.load_pl_df(pl_fname)
else:
    pl_dir = '/scratch/dscho/unit_activity_and_hpc_theta/data/crosselec_phase_locking/phase_locking/unit_to_region'
    pl_files = glob(os.path.join(pl_dir, 'phase_locking_stats*.pkl'))
    pl_df = pd.DataFrame(dio.open_pickle(pl_files[0])).T
    for f in pl_files[1:]:
        pl_df = pl_df.append(dio.open_pickle(f))
    pl_df.reset_index(drop=True, inplace=True)

print(pl_df.shape, pl_df.query("(edge=='ctx-hpc')").shape)

print('Done in {:.2f} secs'.format(time() - start_time))

(13793, 56) (2405, 56)
Done in 14.95 secs


In [257]:
# # Update phase-locking parameters.
# start_time = time()

# n_updated = {'sig': 0, 'nsig': 0}
# for idx, row in pl_df.iterrows():
#     new_pl_params, updated = phase_locking.update_pl_freq_time(pl_df.loc[idx])
    
#     if updated:
#         for key in new_pl_params.keys():
#             pl_df.at[idx, key] = new_pl_params[key]
        
#         if row['sig_z_fdr']:
#             n_updated['sig'] += 1
#         else:
#             n_updated['nsig'] += 1
    
# pl_df['pl_latency'] = np.abs(pl_df['pl_time_shift'])
    
# print('{}/{} ({:.1f}%) significant phase-locking parameters updated'.format(n_updated['sig'], len(pl_df.loc[pl_df['sig_z_fdr']==True]), 
#                                                                             100 * n_updated['sig'] / len(pl_df.loc[pl_df['sig_z_fdr']==True])), end='\n\n')
# print('{}/{} ({:.1f}%) insignificant phase-locking parameters updated'.format(n_updated['nsig'], len(pl_df.loc[pl_df['sig_z_fdr']==False]), 
#                                                                               100 * n_updated['sig'] / len(pl_df.loc[pl_df['sig_z_fdr']==False])), end='\n\n')

# print('Done in {:.2f} secs'.format(time() - start_time))

430/3113 (13.8%) significant phase-locking parameters updated

477/10680 (4.0%) insignificant phase-locking parameters updated

Done in 27.38 secs


In [4]:
def bootstrap_p(x):
        """Return a p-value.
    
        For each connection, the maximum observed Z-score (across frequencies) 
        is compared against the surrogate distribution of maximum Z-scores.
        """
        obs = x.mrls_z # n_freq vec
        null = x.bs_mrls_z # n_freq x n_boot vec
        max_obs = np.max(obs)
        max_null = np.max(null, axis=0)
        n_bootstraps = len(max_null)
        bs_ind = np.sum(max_null >= max_obs)
        pval = (1 + bs_ind) / (1 + n_bootstraps)
        return bs_ind, pval

def phase_offset_inds_uniform(hpc_row,
                              info,
                              iFreq,
                              iTimeLag=0, # number of steps to shift the spike train
                              sampling_rate=2000,
                              time_win=2,
                              n_bootstraps=1000,
                              input_dir='/scratch/dscho/unit_activity_and_hpc_theta/data/crosselec_phase_locking',
                              phase_dir='/scratch/dscho/unit_activity_and_hpc_theta/data/crosselec_phase_locking/wavelet/phase',
                              phase_fname='phase-{}-iChan{}-iFreq{}-{}Hz-notch60_120Hz-nospikeinterp-5cycles-16log10freqs_0.5_to_90.5Hz.pkl'):
    """Calculate the preferred phase and phase offsets by spike for a given unit-to-region comparison."""
        
    # General params.
    subj_sess, unit, lfp_chans = info.subj_sess, info.unit, info.lfp_chan_inds
    lfp_chans = list(np.unique(lfp_chans + [info.unit_chan_ind]))
    sampling_rate = int(sampling_rate)
    cut_inds = int(sampling_rate * time_win)
    
    # Load spike inds and remove time_win secs from the beggining and end of the session.
    spike_inds, n_timepoints = dio.open_pickle(os.path.join(input_dir, 'spike_inds', 'spike_inds-{}Hz-{}-unit{}.pkl'
                                                            .format(sampling_rate, subj_sess, unit)))
    spike_inds = np.unique(spike_inds[(spike_inds>cut_inds) & (spike_inds<(n_timepoints-cut_inds))])
    if iTimeLag != 0:
        spike_inds = shift_spike_inds(spike_inds, n_timepoints, iTimeLag)
        
    # Get the circular distance between each spike phase (at the chosen frequency) 
    # and the mean preferred phase for each channel. Then take the average circular distance
    # across channels.    
    spike_phases = []
    phase_offsets = []
    
    # Recenter phases around 0.
    for iChan in lfp_chans:
        phase = dio.open_pickle(os.path.join(phase_dir, phase_fname.format(subj_sess, iChan, iFreq, sampling_rate)))
        spike_phases_ = phase[spike_inds]
        pref_phase_, mrl_ = circstats.circmoment(spike_phases_)
        spike_phases.append(spike_phases_)
        phase_offsets.append(pycircstat.descriptive.cdiff(spike_phases_, np.repeat(pref_phase_, len(spike_phases_))))
    
    # Get the new spike inds.
    try:
        sel_spikes = get_sel_spikes(spike_phases) # phase_offsets
    except ValueError:
        output = {}
        output['local'] = [0, 0, 0, 1]
        output['hpc'] = [0, 0, 0, 1]
        return output
        
    # Note we get different spike inds for each LFP channel now.
    spike_inds = np.array([spike_inds[sel_spikes[iChan, :]] for iChan in range(sel_spikes.shape[0])]) 
    
    # -----------------------------
    bs_steps = dio.open_pickle(os.path.join(input_dir, 'bootstrap_shifts', '{}_{}bootstraps_{}Hz_{}timewin.pkl'
                                            .format(subj_sess, int(n_bootstraps), int(sampling_rate), int(time_win))))
    alpha = 0.05
    output = {}
    
    # Local
    mrls_arr = []
    bs_mrls_arr = []
    for ii, iChan in enumerate(lfp_chans):
        phase = dio.open_pickle(os.path.join(phase_dir, phase_fname.format(subj_sess, iChan, iFreq, sampling_rate)))
        spike_phases_ = phase[spike_inds[ii, :]]
        mrls_arr.append(circstats.circmoment(spike_phases_)[1])
        
        bs_mrls_arr_ = []
        for step in bs_steps:
            spike_inds_shifted = shift_spike_inds(spike_inds[ii, :], n_timepoints, step)
            spike_phases_ = phase[spike_inds_shifted]
            bs_mrls_arr_.append(circstats.circmoment(spike_phases_)[1])
        bs_mrls_arr.append(bs_mrls_arr_)
    n_spikes = spike_inds.shape[1]
    obs_mrl = np.mean(mrls_arr) # scalar
    bs_mrls_arr = np.mean(bs_mrls_arr, axis=0) # n_boot
    bs_mrls_arr_z = stats.zscore(bs_mrls_arr)
    obs_mrl_z = (obs_mrl - np.mean(bs_mrls_arr)) / np.std(bs_mrls_arr)
    p_val = (1 + np.sum(bs_mrls_arr_z >= obs_mrl_z)) / (1 + n_bootstraps)
    output['local'] = [n_spikes, obs_mrl, obs_mrl_z, p_val]
    
    # HPC
    # Note: a random spike subset is used for each hippocampal channel.
    mrls_arr = []
    bs_mrls_arr = []
    if len(hpc_row.lfp_chan_inds) > spike_inds.shape[0]:
        lfp_chan_inds = np.random.choice(hpc_row.lfp_chan_inds, spike_inds.shape[0], replace=False)
    else:
        lfp_chan_inds = hpc_row.lfp_chan_inds
    for ii, iChan in enumerate(lfp_chan_inds):
        phase = dio.open_pickle(os.path.join(phase_dir, phase_fname.format(subj_sess, iChan, iFreq, sampling_rate)))
        spike_phases_ = phase[spike_inds[ii, :]]
        mrls_arr.append(circstats.circmoment(spike_phases_)[1])
        
        bs_mrls_arr_ = []
        for step in bs_steps:
            spike_inds_shifted = shift_spike_inds(spike_inds[ii, :], n_timepoints, step)
            spike_phases_ = phase[spike_inds_shifted]
            bs_mrls_arr_.append(circstats.circmoment(spike_phases_)[1])
        bs_mrls_arr.append(bs_mrls_arr_)
    n_spikes = spike_inds.shape[1]
    obs_mrl = np.mean(mrls_arr) # scalar
    bs_mrls_arr = np.mean(bs_mrls_arr, axis=0) # n_boot
    bs_mrls_arr_z = stats.zscore(bs_mrls_arr)
    obs_mrl_z = (obs_mrl - np.mean(bs_mrls_arr)) / np.std(bs_mrls_arr)
    p_val = (1 + np.sum(bs_mrls_arr_z >= obs_mrl_z)) / (1 + n_bootstraps)
    output['hpc'] = [n_spikes, obs_mrl, obs_mrl_z, p_val]
    
    return output
    
def get_sel_spikes(spike_phases):
    """Select a subset of spikes (separately for each channel) that will enforce uniformity."""
    n_bins = 6
    bins = np.linspace(-np.pi, np.pi, n_bins+1)
    spike_phases = np.array(spike_phases)
    phase_bins = np.reshape([get_phase_bin(x, bins) for x in spike_phases.flatten()], spike_phases.shape)
    draw_nspikes = np.min([np.min(np.unique(phase_bins[iChan, :], return_counts=True)[1]) for iChan in range(phase_bins.shape[0])])
    sel_spikes = np.array([[np.random.choice(np.where(phase_bins[iChan, :]==iBin)[0], draw_nspikes, replace=False) 
                            for iBin in range(n_bins)]
                           for iChan in range(phase_bins.shape[0])])
    shp = sel_spikes.shape
    sel_spikes = sel_spikes.reshape((shp[0], np.prod(shp[1:]))) # n_chan x draw_nspikes
    return sel_spikes

def get_phase_bin(phi, bins):
    """Return the phase bin."""
    for i in range(len(bins)-1):
        if bins[i] <= phi < bins[i+1]:
            return i
        
def shift_spike_inds(spike_inds, n_timepoints, step):
    """Return the time-shifted spike_inds array.
    
    Parameters
    ----------
    spike_inds : np.ndarray
        Array of spike time indices.
    n_timepoints : int
        Number of timepoints in the recording session.
    step : int
        Number of timepoints to shift the spike train by.
    """
    spike_inds_shifted = spike_inds + step
    
    if step < 0:
        roll_by = -len(spike_inds_shifted[spike_inds_shifted<0])
        spike_inds_shifted[spike_inds_shifted<0] = spike_inds_shifted[spike_inds_shifted<0] + n_timepoints
    else:
        roll_by = len(spike_inds_shifted[spike_inds_shifted>=n_timepoints])
        spike_inds_shifted[spike_inds_shifted>=n_timepoints] = spike_inds_shifted[spike_inds_shifted>=n_timepoints] - n_timepoints

    spike_inds_shifted = np.roll(spike_inds_shifted, roll_by)

    return spike_inds_shifted

In [21]:
subj_sess = hpc_pl_df.iloc[269].subj_sess
unit = hpc_pl_df.iloc[269].unit
pl_df.query("(subj_sess=='{}') & (unit=={}) & (same_hemroi==True)".format(subj_sess, unit)).iloc[0]
output[269]

''

In [31]:
ii

269

In [33]:
# For all cortical neurons that are phase-locked to HPC LFP,
# get the preferred phase and phase offset for each spike when
# referenced to the local LFP at the HPC LFP preferred frequency.
# This info is appended to the unit-to-region phase-locking Series 
# and saved to /.../unit_to_region_local_uniformity
start_time = time()

overwrite = False
freq_col = 'locked_freq_ind_z'

#hpc_pl_df = pl_df.query("(edge=='ctx-hpc') & (same_hem==True) & (sig_z_fdr==True)").reset_index(drop=True).copy()
#output = ['' for x in range(len(hpc_pl_df))]
ii = 269 #0
for index, hpc_row in hpc_pl_df.iloc[ii:].iterrows():
    subj_sess = hpc_row.subj_sess
    unit = hpc_row.unit
    lfp_roi = hpc_row.lfp_hemroi
    hpc_locked_iFreq = hpc_row[freq_col]
    
    local_row = pl_df.query("(subj_sess=='{}') & (unit=={}) & (same_hemroi==True)".format(subj_sess, unit)).iloc[0]
    output[ii] = phase_offset_inds_uniform(hpc_row, local_row, hpc_locked_iFreq) # n_spikes, obs_mrl, obs_mrl_z, p_val)
    ii += 1
                  
hpc_pl_df['local_uniformity'] = output

# Save.
fname = os.path.join('/scratch/dscho/unit_activity_and_hpc_theta/data/crosselec_phase_locking/phase_locking/unit_to_region_local_uniformity',
                     'local_uniformity_results_{}.pkl'.format(freq_col))
dio.save_pickle(hpc_pl_df, fname)
    
print('Done in {:.2f} secs'.format(time() - start_time))

Saved /scratch/dscho/unit_activity_and_hpc_theta/data/crosselec_phase_locking/phase_locking/unit_to_region_local_uniformity/local_uniformity_results_locked_freq_ind_z.pkl
Done in 776.78 secs


In [6]:
freq_col = 'locked_freq_ind_z'
fname = os.path.join('/scratch/dscho/unit_activity_and_hpc_theta/data/crosselec_phase_locking/phase_locking/unit_to_region_local_uniformity',
                     'local_uniformity_results_{}.pkl'.format(freq_col))
hpc_pl_df = dio.open_pickle(fname)
print(hpc_pl_df.shape)

(362, 57)


In [7]:
local_pvals = []
hpc_pvals = []
for idx, row in hpc_pl_df.iterrows():
    local_pvals.append(row['local_uniformity']['local'][3])
    hpc_pvals.append(row['local_uniformity']['hpc'][3])
    
# FDR correct p-values.
alpha = 0.05
local_sig_fdr = sm.stats.multipletests(local_pvals, alpha=alpha, method='fdr_bh', is_sorted=False, returnsorted=False)[0]
hpc_sig_fdr = sm.stats.multipletests(hpc_pvals, alpha=alpha, method='fdr_bh', is_sorted=False, returnsorted=False)[0]

print('{}/{} ({:.2f}%) local phase-locking after subsampling correction'.format(np.sum(local_sig_fdr), len(local_sig_fdr), 100 * np.sum(local_sig_fdr)/len(local_sig_fdr)))
print('{}/{} ({:.2f}%) local phase-locking after subsampling correction'.format(np.sum(hpc_sig_fdr), len(hpc_sig_fdr), 100 * np.sum(hpc_sig_fdr)/len(hpc_sig_fdr)))

0/362 (0.00%) local phase-locking after subsampling correction
334/362 (92.27%) local phase-locking after subsampling correction


In [34]:
hpc_pl_df['locked_mrl_z_adj'] = hpc_pl_df['local_uniformity'].apply(lambda x: x['hpc'][2])
hpc_pl_df['post_min_pre'] = hpc_pl_df.apply(lambda x: x['local_uniformity']['hpc'][2] - x['locked_mrl_z'], axis=1)

In [56]:
# Create pre- and post-correction phase-locking strength df for MEM.
df1 = hpc_pl_df[['subj', 'subj_sess_unit', 'locked_mrl_z']].copy()
df1['grp'] = np.repeat('pre', len(hpc_pl_df))
df2 = hpc_pl_df[['subj', 'subj_sess_unit', 'locked_mrl_z_adj']].copy()
df2.rename(columns={'locked_mrl_z_adj': 'locked_mrl_z'}, inplace=True)
df2['grp'] = np.repeat('post', len(hpc_pl_df))
df = pd.concat((df1, df2), axis=0)

# Save.
fname = os.path.join('/scratch/dscho/unit_activity_and_hpc_theta/data/crosselec_phase_locking/phase_locking/unit_to_region_local_uniformity',
                     'pl_strength-pre_vs_post_local_uniformity_correction-{}.csv'.format(freq_col))
df.to_csv(fname, index=False)
print('Saved {}'.format(fname))

Saved /scratch/dscho/unit_activity_and_hpc_theta/data/crosselec_phase_locking/phase_locking/unit_to_region_local_uniformity/pl_strength-pre_vs_post_local_uniformity_correction-locked_freq_ind_z.csv


In [57]:
arr = hpc_pl_df.apply(lambda x: x['local_uniformity']['hpc'][2] - x['locked_mrl_z'], axis=1)
print('{:.1f} +/- {:.1f} change in phase-locking strength after subsampling correction: {}'
      .format(np.mean(arr), np.std(arr), stats.ttest_rel(hpc_pl_df['local_uniformity'].apply(lambda x: x['hpc'][2]), hpc_pl_df['locked_mrl_z'])))

-1.5 +/- 2.1 change in phase-locking strength after subsampling correction: Ttest_relResult(statistic=-13.03117094697981, pvalue=4.427911292426011e-32)


In [58]:
stats.ttest_1samp(hpc_pl_df['post_min_pre'], 0)

Ttest_1sampResult(statistic=-13.03117094697981, pvalue=4.427911292426011e-32)

In [40]:
# Perform a paired-samples t-test across neurons within each subject,
# than a one-sample t-test on the t-stats from each subject (in
# subjects with > n_neuron_thresh neurons).
n_neuron_thresh = 7
subj_t_stats = hpc_pl_df.groupby('subj').agg({'unit': len, 'post_min_pre': lambda x: stats.ttest_1samp(x, 0)[0]}).query("unit>{}".format(n_neuron_thresh))['post_min_pre']
mod_fit = stats.ttest_1samp(subj_t_stats, 0)
print('{:.1f} +/- {:.1f}; t({}) = {:.1f}, p = {:.4f}'.format(np.mean(subj_t_stats), np.std(subj_t_stats), len(subj_t_stats)-1, mod_fit[0], mod_fit[1]))

-4.5 +/- 1.2; t(13) = -13.6, p = 0.0000


In [213]:
hpc_pl_df['sig_fdr_locun'] = hpc_sig_fdr

In [16]:
hpc_pl_df.head()

,subj,sess,subj_sess,subj_sess_unit,subj_unit_chan,unit,unit_chan_ind,unit_hemroi,unit_hem,unit_roi,unit_is_hpc,unit_nspikes,unit_fr,lfp_hemroi,lfp_hem,lfp_roi,lfp_is_hpc,same_chan,same_hemroi,same_hem,same_roi,both_hpc,unit_hemroi2,lfp_hemroi2,unit_roi2,lfp_roi2,same_roi2,lfp_chan_inds,edge,unit_nsamp_spikes,spike_inds,bs_mrls,tl_mrls,bs_mrls_z,tl_mrls_z,mrls_z,locked_freq_ind_z,locked_mrl_z,bs_ind_z,bs_pval_z,sig_z,tl_locked_freq_z,tl_locked_time_z,tl_locked_mrl_z,pref_phase,phase_offsets,pref_phase_tl_locked_time_freq_z,phase_offsets_tl_locked_time_freq_z,unit_roi3,roi,roi_unit_to_lfp,pl_freq,pl_strength,pl_time_shift,pl_latency,sig_z_fdr,local_uniformity,post_min_pre
0,U387,3,U387_ses3,U387_ses3_6,U387_21,6,20,LA,L,A,0,423,0.231980,LAH,L,AH,True,False,False,True,False,False,lamy,lhpc,amy,hpc,False,"[32, 33, 34, 35, 36, 37, 38, 39]",ctx-hpc,423,"[9508, 9518, 11014, 11037, 11062, 15825, 51326...","[[0.05219674, 0.084073305, 0.06320857, 0.08092...","[[0.09039113, 0.090184204, 0.08998255, 0.08980...","[[-1.1723703, 1.3166379, -0.3125368, 1.0708197...","[[-2.0992706, -2.0899472, -2.0801947, -2.06965...","[0.8707276, 1.4436296, 4.008028, 3.0005815, -0...",8,4.724211,2,0.002997,True,8,10,4.758540,-2.913358,"[2.3581858, 2.5154085, 0.23363686, 0.71599644,...",-2.913358,"[2.3581858, 2.5154085, 0.23363686, 0.71599644,...",amy,amy,amy_ipsi,8,4.758540,10.0,10.0,True,"{'local': [306, 0.017439248988637714, -2.21497...",-0.050306
1,U387,3,U387_ses3,U387_ses3_3,U387_19,3,18,LA,L,A,0,1562,0.851861,LAH,L,AH,True,False,False,True,False,False,lamy,lhpc,amy,hpc,False,"[32, 33, 34, 35, 36, 37, 38, 39]",ctx-hpc,1562,"[4972, 5314, 7379, 7398, 7421, 7760, 7814, 911...","[[0.028890362, 0.027846709, 0.022980802, 0.032...","[[0.026269047, 0.02618736, 0.026093392, 0.0258...","[[0.42256653, 0.22273299, -0.7089669, 1.055844...","[[0.27469975, 0.2739326, 0.27459526, 0.2788782...","[1.8227459, 5.11836, 6.1532187, 6.4382844, 4.4...",3,6.438284,0,0.000999,True,1,-790,7.434636,-2.440690,"[1.1787579, 0.7598909, 2.292517, 2.2141814, 2....",2.593729,"[0.8038979, 0.9483306, 0.97260374, 0.9282509, ...",amy,amy,amy_ipsi,3,6.640398,80.0,80.0,True,"{'local': [1110, 0.01262555266995801, -2.06839...",-0.267365
2,U387,3,U387_ses3,U387_ses3_26,U387_61,26,60,LOF,L,OF,0,4969,2.704986,LAH,L,AH,True,False,False,True,False,False,lofc,lhpc,ofc,hpc,False,"[32, 33, 34, 35, 36, 37, 38, 39]",ctx-hpc,4969,"[5272, 6011, 6554, 9247, 9306, 9559, 10694, 11...","[[0.014340243, 0.0126502095, 0.011336942, 0.01...","[[0.016440298, 0.016464954, 0.016543264, 0.016...","[[-0.23964491, -0.8017613, -1.2385627, 0.95838...","[[2.0517569, 2.0849283, 2.1190915, 2.1648436, ...","[2.309051, 4.954284, 2.2507865, 1.0187025, 1.6...",1,4.954284,0,0.000999,True,9,-1040,5.714668,3.082371,"[1.665881, 1.6320474, 1.0249296, 0.7529751, 0....",-2.774867,"[2.4153926, 1.1803647, 2.3396852, 0.58993965, ...",ctx,ctx,ctx_ipsi,1,4.977881,30.0,30.0,True,"{'local': [4584, 0.0028049642757618553, -3.485...",-0.430723
3,U387,3,U387_ses3,U387_ses3_1,U387_17,1,16,LA,L,A,0,18722,10.199519,LAH,L,AH,True,False,False,True,False,False,lamy,lhpc,amy,hpc,False,"[32, 33, 34, 35, 36, 37, 38, 39]",ctx-hpc,18722,"[4080, 4161, 4212, 4238, 4694, 4809, 5060, 515...","[[0.014319505, 0.010832321, 0.01219813, 0.0100...","[[0.01265388, 0.012578488, 0.012501087, 0.0124...","[[1.029666, -0.4625996, 0.12186901, -0.7964291...","[[2.1199653, 2.0494995, 1.9880918, 1.9304359, ...","[2.953062, 7.3948736, 8.465768, 7.050589, 12.5...",4,12.505577,0,0.000999,True,4,120,13.454884,-1.806968,"[1.9973028, 2.0383775, 1.944145, 1.8529598, 1....",-3.039032,"[1.8458941, 2.1235297, 2.169243, 2.0930512, 1....",amy,amy,amy_ipsi,4,13.454884,120.0,120.0,True,"{'local': [16116, 0.004638156082003941, -1.303...",-4.215973
4,U387,3,U387_ses3,U387_ses3_2,U387_18,2,17,LA,L,A,0,5359,2.918494,LAH,L,AH,True,False,False,True,False,False,lamy,lhpc,amy,hpc,False,"[32, 33, 34, 35, 36, 37, 38, 39]",ctx-hpc,5359,"[7360, 7397, 7462, 7771, 7783, 7795, 7813, 784...","[[0.021809602, 